# 第19章 Python并发模型

## 19.1 核心概念

- 并发：处理多个待定认为，一次处理一个或并行处理多个，直到所有任务最终都成功或失败。并发也叫做多任务处理。
- 并行：同时执行多个计算任务的能力。
- 执行单元：并发执行代码的对象的统称，每个对象的状态和调用栈是独立的，Python原生支持3种执行单元：进程、线程和协程。
- 进程：
   - 计算机程序运行时的一个实例，消耗内存和部分CPU时间。
   - 进程通过管道、套接字或内存映射文件进行通信。
   - 进程可以派生子进程，子进程之间以及与父进程之间是隔离的。
   - 进程支持抢占式多任务处理机制：操作系统调度程序定期抢占（挂起）运行中的进程，让其他进程运行。
- 线程：
   - 单个进程中的执行单元。
   - 通过调用操作系统API，进程可以创建更多线程，执行并发操作。
   - 一个进程内的线程共享相同的内存空间。
- 协程：
   - 可以挂起自身并在以后恢复的函数。
   - 经典协程由生成器函数构建，原生协程使用`async def`定义。
   - 协程支持协作式多任务处理：一个协程必须使用`yield`或`await`关键字显式放弃控制权，另一个协程才可以并发开展工作。
- 队列：一种数据结构，可以放入和取出项，顺序通常是先入先出（FIFO）。独立的执行单元可以通过队列交换引用数据和控制消息。
- 锁：一种供执行单元用来同步操作和避免数据损坏的对象。
- 争用：对有限资源的争夺。当多个执行单元尝试访问共享资源时，就会发生资源争用。

## 19.2 并发的"Hello World"示例

功能：启动一个函数，阻塞3秒，期间在界面上会显示一个旋转的指针动画，当计算结束后，显示结果：`Answer: 42`。

### 19.2.1 使用线程实现旋转指针

In [1]:
import itertools
import time
from threading import Thread, Event

def spin(msg: str, done: Event) -> None:
    # 循环展示指针字符
    for char in itertools.cycle(r'\|/-'):  
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        # 线程阻塞，帧数为10fps
        if done.wait(.1):
            break  
    blanks = ' ' * len(status)
    # 将光标移动到开头，清空状态行
    print(f'\r{blanks}\r', end='')  

def slow() -> int:
    # 停止3秒
    time.sleep(3)
    return 42

In [2]:
def supervisor() -> int:
    done = Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    # 启动线程
    spinner.start()
    # 调用slow，阻塞main线程
    result = slow()
    # 终止spin函数的for循环
    done.set()
    # 等待，直到spinner线程结束
    spinner.join()
    return result

In [3]:
result = supervisor()
print(f'Answer: {result}')

spinner object: <Thread(Thread-5 (spin), initial)>
Answer: 42  


### 19.2.2 使用进程实现旋转指针

In [4]:
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize

def spin(msg: str, done: synchronize.Event) -> None:
    # 循环展示指针字符
    for char in itertools.cycle(r'\|/-'):  
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        # 线程阻塞，帧数为10fps
        if done.wait(.1):
            break  
    blanks = ' ' * len(status)
    # 将光标移动到开头，清空状态行
    print(f'\r{blanks}\r', end='')  

def slow() -> int:
    # 停止3秒
    time.sleep(3)
    return 42

In [5]:
def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    # 启动线程
    spinner.start()
    # 调用slow，阻塞main线程
    result = slow()
    # 终止spin函数的for循环
    done.set()
    # 等待，直到spinner线程结束
    spinner.join()
    return result

In [6]:
result = supervisor()
print(f'Answer: {result}')

spinner object: <Process name='Process-1' parent=12812 initial>
Answer: 42


### 19.2.3 使用协程实现旋转指针

In [7]:
import itertools
import asyncio

async def spin(msg: str) -> None:
    # 循环展示指针字符
    for char in itertools.cycle(r'\|/-'):  
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        try:
            # 线程阻塞，帧数为10fps
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    blanks = ' ' * len(status)
    # 将光标移动到开头，清空状态行
    print(f'\r{blanks}\r', end='')  

async def slow() -> int:
    # 停止3秒
    await asyncio.sleep(3)
    return 42

In [8]:
async def supervisor() -> int:
    # 调度spin最终执行
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')  
    # 调用slow，阻塞supervisor，直到slow返回
    result = await slow()
    # 抛出CancelledError异常
    spinner.cancel()
    return result

In [9]:
result = await supervisor()
print(f'Answer: {result}')

spinner object: <Task pending name='Task-6' coro=<spin() running at C:\Users\hurui\AppData\Local\Temp\ipykernel_12812\3509005970.py:4>>
- thinking!Answer: 42


- asyncio.run(coro())：在常规函数中调用，驱动一个协程对象，通常作为程序中所有异步代码的入口。
- asyncio.create_task(coro())：在协程中调用，调度另一个协程最终执行。
- await coro()：在协程中调用，把控制权转给coro()返回的协程对象，中止当前协程，直到coro的主体返回。

## 19.3 自建进程池

### 19.3.1 第1版：素数检测

In [10]:
import math

PRIME_FIXTURE = [
    (2, True),
    (142702110479723, True),
    (299593572317531, True),
    (3333333333333301, True),
    (3333333333333333, False),
    (3333335652092209, False),
    (4444444444444423, True),
    (4444444444444444, False),
    (4444444488888889, False),
    (5555553133149889, False),
    (5555555555555503, True),
    (5555555555555555, False),
    (6666666666666666, False),
    (6666666666666719, True),
    (6666667141414921, False),
    (7777777536340681, False),
    (7777777777777753, True),
    (7777777777777777, False),
    (9999999999999917, True),
    (9999999999999999, False),
]

NUMBERS = [n for n, _ in PRIME_FIXTURE]

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

In [11]:
from time import perf_counter
from typing import NamedTuple

class Result(NamedTuple):  # <1>
    prime: bool
    elapsed: float

def check(n: int) -> Result:
    # 检查是否为素数
    t0 = perf_counter()
    prime = is_prime(n)
    return Result(prime, perf_counter() - t0)

def main() -> None:
    print(f'Checking {len(NUMBERS)} numbers sequentially:')
    t0 = perf_counter()
    for n in NUMBERS:
        prime, elapsed = check(n)
        label = 'P' if prime else ' '
        print(f'{n:16}  {label} {elapsed:9.6f}s')

    elapsed = perf_counter() - t0  # <4>
    print(f'Total time: {elapsed:.2f}s')

In [12]:
main()

Checking 20 numbers sequentially:
               2  P  0.000001s
 142702110479723  P  0.278544s
 299593572317531  P  0.391379s
3333333333333301  P  1.342821s
3333333333333333     0.000012s
3333335652092209     1.319537s
4444444444444423  P  1.545259s
4444444444444444     0.000002s
4444444488888889     1.485832s
5555553133149889     1.711040s
5555555555555503  P  1.662344s
5555555555555555     0.000006s
6666666666666666     0.000000s
6666666666666719  P  1.868928s
6666667141414921     1.840363s
7777777536340681     2.034407s
7777777777777753  P  1.990972s
7777777777777777     0.000004s
9999999999999917  P  2.309609s
9999999999999999     0.000004s
Total time: 19.79s


### 19.3.2 第2版：基于进程的方案

**改进方案：** 使用多个进程把素数检测分配给多个CPU核。

In [13]:
import sys
from time import perf_counter
from typing import NamedTuple
from multiprocessing import Process, SimpleQueue, cpu_count  
from multiprocessing import queues

class PrimeResult(NamedTuple):
    n: int
    prime: bool
    elapsed: float

# 建立任务队列        
JobQueue = queues.SimpleQueue[int]
ResultQueue = queues.SimpleQueue[PrimeResult]

def check(n: int) -> PrimeResult:
    t0 = perf_counter()
    res = is_prime(n)
    return PrimeResult(n, res, perf_counter() - t0)

def worker(jobs: JobQueue, results: ResultQueue) -> None:
    # 取出要检查的数
    while n := jobs.get():
        # 存放结果
        results.put(check(n))      
    results.put(PrimeResult(0, False, 0.0))

def start_jobs(procs: int, jobs: JobQueue, results: ResultQueue) -> None:
    # 初始化任务队列
    for n in NUMBERS:
        jobs.put(n)
    # 创建多个进程    
    for _ in range(procs):
        proc = Process(target=worker, args=(jobs, results))  
        proc.start()  
        jobs.put(0)

def main(count=-1) -> None:
    if cpu_count != -1:
        procs = count
    else:
        procs = cpu_count()
    
    print(f'Checking {len(NUMBERS)} numbers with {procs} processes:')
    t0 = perf_counter()
    jobs: JobQueue = SimpleQueue()  
    results: ResultQueue = SimpleQueue()
    # 启动任务    
    start_jobs(procs, jobs, results)  
    checked = report(procs, results)
    elapsed = perf_counter() - t0
    print(f'{checked} checks in {elapsed:.2f}s')  

def report(procs: int, results: ResultQueue) -> int: 
    checked = 0
    procs_done = 0
    # 在所有进程都结束之前一直循环
    while procs_done < procs:
        n, prime, elapsed = results.get()
        if n == 0:  # <9>
            procs_done += 1
        else:
            checked += 1
            label = 'P' if prime else ' '
            print(f'{n:16}  {label} {elapsed:9.6f}s')
    return checked

## 19.4 Python的多核应用

- 系统管理：Pyhton广泛用于管理大型服务器集群、路由器、负载均衡程序和网络接入存储。使用Python脚本发送命令，由远程设备执行，实现配置任务自动化。
- 数据科学：
    - Jupyter项目：提供两个基于游览器的界面（Jupyter Notebook和JupyterLab），供用户运行和记录可在远程计算机上通过网络运行的分析代码。
    - TensorFlow和PyTorch：深度学习框架。
    - Dask：并行计算库，可以把工作委托给本地进程或设备集群。可以在JupyterLab或Jupyter Notebook中使用，利用Boken不仅可以实现数据可视化，还可以实现交互式仪表板，以接近实时的方式显示数据流和跨进程、跨设备的计算。
- 服务器端Web和移动开发：Python广泛用于Web应用程序开发和为移动应用程序提供支持的后端API开发。
    - 一个应用程序服务器，在Python应用程序的多个实例之间分配负载。
    - 一个任务队列，提供易于使用的高级API，为设备中运行的进程分配任务。
- WSGI应用程序服务器：WSGI是Python框架或应用程序接收HTTP服务器请求并向其发送响应的标准API。
    

## 19.5 任务队列

任务队列的用途：
- 运行后台作业。
- 在Web请求完成后执行某些操作。
- 异步执行某些操作，失败后重试，确保操作一定完成。
- 安排定期工作。

任务队列的特点：支持横向伸缩。

## 19.6 杂谈

线程和锁模式的不受约束性：
- 线程可以共享访问任意可变的数据结构。
- 调度程序几乎可以在任何时刻中断线程。
- 锁往往是建议性的，在更新共享的数据结构之前，必须显式持有锁。

参与者模式的约束性：
- 参与者可以有内部状态，但是不能与其它参与者分享状态。
- 参与者之间通过收发消息进行通信。
- 消息中只能存有数据的副本，不能引用可变的数据。
- 一个参与者一次只处理一个消息。对于单个参与者，没有并发执行的概念。